In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import traceback

# ==============================================================================
# 1. 配置层 (Configuration Layer)
# ==============================================================================
class Config:
    """集中管理所有文件路径和常量"""
    # --- 输入输出文件 ---
    INPUT_FILE = Path(r'E:\重点行业top20_全行业.xlsx')  # <--- 修改为你的源文件路径
    OUTPUT_FILE = Path(r'E:\重点行业top20_结果.xlsx') # <--- 修改为你的输出文件路径

    # --- 特殊Sheet名称 ---
    INDUSTRY_SUMMARY_SHEET = '行业' # 提供计算数据的“字典”Sheet

    # --- 源数据列名 ---
    SRC_CUST_NAME = 'cust_name'
    SRC_ORG_NAME = 'mgt_org_name'
    SRC_ORG_CODE = 'mgt_org_code'  # 【新增】组织机构代码列
    SRC_CURRENT_MONTH_AVG = '202509日均'
    SRC_PREVIOUS_YEAR_AVG = '202409日均'
    SRC_YOY_CHANGE = '同比' # Year-over-Year, 同比
    SRC_MOM_CHANGE = '环比' # Month-over-Month, 环比

    # --- 目标数据列名 ---
    TGT_CUST_NAME = '用户名称'
    TGT_CITY = '所属地市'
    TGT_CURRENT_MONTH_AVG = '202509日均'
    TGT_YOY_CHANGE = '同比'
    TGT_MOM_CHANGE = '环比'
    TGT_INDUSTRY_SHARE = '占行业比重'
    TGT_CONTRIBUTION_RATE = '贡献率'
    
    # --- “行业”Sheet中的列名 ---
    # 假设“行业”Sheet有以下列，请根据你的实际情况修改
    INDUSTRY_NAME_COL = '行业名称'          # <--- “行业”Sheet中，行业名字所在的列
    INDUSTRY_AVG_LOAD_COL = '日均售电量'    # <--- “行业”Sheet中，行业总电量所在的列
    INDUSTRY_YOY_COL = '同比增长'          # <--- “行业”Sheet中，行业同比增长率所在的列

    # 【新增】地市代码映射方法
    @staticmethod
    def get_city_mapping() -> dict:
        """
        返回组织机构代码前5位到地市名称的映射字典。
        【请在这里补充和修改你的地市代码】
        """
        city_map = {
            '42409': '宜昌',
            '42401': '武汉', 
            '42402': '黄石', 
            '42403': '十堰', 
            '42404': '襄阳',
            '42405': '黄冈',
            '42406': '鄂州',
            '42411': '荆门', 
            '42408': '孝感', 
            '42407': '荆州', 
            '42410': '咸宁', 
            '42412': '随州', 
        }
        return city_map

# ==============================================================================
# 2. 数据处理与计算层 (Processing & Calculation Layer)
# ==============================================================================

def robust_to_numeric(series: pd.Series) -> pd.Series:
    """
    一个健壮的函数，用于将可能包含'\\N'等文本的列转换为数值类型。
    """
    # 【已修复SyntaxError】使用双反斜杠 '\\N' 来明确表示要替换的字符串
    s = series.replace('\\N', np.nan, regex=False).replace('nan', np.nan)
    return pd.to_numeric(s, errors='coerce')

def map_org_code_to_city(org_code_series: pd.Series, mapping: dict) -> pd.Series:
    """
    根据组织机构代码的前5位，将其映射到对应的地市名称。
    """
    # 确保代码列是字符串类型，然后提取前5位
    codes_5_digits = org_code_series.astype(str).str[:5]
    
    # 使用.map()方法进行高效映射，对于找不到的编码填充为'未知地市'
    city_names = codes_5_digits.map(mapping).fillna('未知地市')
    
    return city_names

def calculate_metrics(df: pd.DataFrame, industry_data: pd.Series) -> pd.DataFrame:
    """
    在单个行业的DataFrame上执行所有指标计算。
    【已修正】根据最终要求调整了“占行业比重”和“贡献率”的计算公式。
    """
    # --- 提取行业级别的数据 ---
    industry_total_avg_load = industry_data[Config.INDUSTRY_AVG_LOAD_COL]
    industry_yoy_growth = industry_data[Config.INDUSTRY_YOY_COL] / 100.0

    # --- 准备用户级别的数据 ---
    cust_current_month_avg = df[Config.SRC_CURRENT_MONTH_AVG]
    cust_previous_year_avg = df[Config.SRC_PREVIOUS_YEAR_AVG]

    # --- 开始计算 ---
    
    # 1. 占行业比重
    # 【已修正】在原公式基础上，增加了除以 10000 的缩放
    df[Config.TGT_INDUSTRY_SHARE] = ((cust_current_month_avg / 10000) / industry_total_avg_load) * 100

    # 2. 贡献率
    numerator = cust_current_month_avg - cust_previous_year_avg
    denominator = industry_total_avg_load - (industry_total_avg_load / (1 + industry_yoy_growth))
    
    if np.isclose(denominator, 0):
        df[Config.TGT_CONTRIBUTION_RATE] = np.nan
    else:
        # 【已修正】在原公式基础上，增加了除以 1000000 的缩放
        base_contribution_rate = (numerator / denominator) * 100
        df[Config.TGT_CONTRIBUTION_RATE] = base_contribution_rate / 1000000

    return df

def process_single_sheet(df: pd.DataFrame, industry_data: pd.Series) -> pd.DataFrame:
    """
    对从单个Sheet读取的DataFrame进行完整的转换、计算和格式化。
    【已更新】使用精确的编码映射代替粗糙的字符串截取来生成“所属地市”。
    """
    # 1. 清洗和转换数值列
    numeric_cols = [
        Config.SRC_CURRENT_MONTH_AVG, Config.SRC_PREVIOUS_YEAR_AVG,
        Config.SRC_YOY_CHANGE, Config.SRC_MOM_CHANGE
    ]
    for col in numeric_cols:
        df[col] = robust_to_numeric(df[col])

    # 2. 计算核心指标
    df = calculate_metrics(df, industry_data)

    # 3. 选择并重命名列
    city_mapping = Config.get_city_mapping()
    
    result_df = pd.DataFrame({
        Config.TGT_CUST_NAME: df[Config.SRC_CUST_NAME],
        Config.TGT_CITY: map_org_code_to_city(df[Config.SRC_ORG_CODE], city_mapping),
        Config.TGT_CURRENT_MONTH_AVG: df[Config.SRC_CURRENT_MONTH_AVG] / 10000,
        Config.TGT_YOY_CHANGE: df[Config.SRC_YOY_CHANGE],
        Config.TGT_MOM_CHANGE: df[Config.SRC_MOM_CHANGE],
        Config.TGT_INDUSTRY_SHARE: df[Config.TGT_INDUSTRY_SHARE],
        Config.TGT_CONTRIBUTION_RATE: df[Config.TGT_CONTRIBUTION_RATE]
    })

    # 4. 格式化输出
    format_cols = [
        Config.TGT_CURRENT_MONTH_AVG, Config.TGT_YOY_CHANGE, Config.TGT_MOM_CHANGE,
        Config.TGT_INDUSTRY_SHARE, Config.TGT_CONTRIBUTION_RATE
    ]
    result_df[format_cols] = result_df[format_cols].round(2)

    return result_df

# ==============================================================================
# 3. 主流程 (Main Workflow)
# ==============================================================================
def main():
    """主执行函数"""
    print(f"开始处理文件: {Config.INPUT_FILE}")
    
    if not Config.INPUT_FILE.exists():
        print(f"[致命错误] 输入文件不存在: {Config.INPUT_FILE}")
        return

    try:
        print(f"正在加载“{Config.INDUSTRY_SUMMARY_SHEET}”Sheet作为计算依据...")
        industry_summary_df = pd.read_excel(Config.INPUT_FILE, sheet_name=Config.INDUSTRY_SUMMARY_SHEET)
        
        summary_numeric_cols = [Config.INDUSTRY_AVG_LOAD_COL, Config.INDUSTRY_YOY_COL]
        for col in summary_numeric_cols:
            industry_summary_df[col] = robust_to_numeric(industry_summary_df[col])
        
        industry_summary_df = industry_summary_df.set_index(Config.INDUSTRY_NAME_COL)
        print("加载成功！")

        Config.OUTPUT_FILE.parent.mkdir(parents=True, exist_ok=True)
        writer = pd.ExcelWriter(Config.OUTPUT_FILE, engine='openpyxl')
        
        xls = pd.ExcelFile(Config.INPUT_FILE)
        sheet_names_to_process = [name for name in xls.sheet_names if name != Config.INDUSTRY_SUMMARY_SHEET]

        print("\n开始批量处理各个行业Sheet...")
        for sheet_name in sheet_names_to_process:
            print(f"  -> 正在处理Sheet: '{sheet_name}'")
            try:
                if sheet_name in industry_summary_df.index:
                    industry_data = industry_summary_df.loc[sheet_name]
                else:
                    print(f"    [警告] 在“{Config.INDUSTRY_SUMMARY_SHEET}”Sheet中找不到行业'{sheet_name}'的数据，已跳过。")
                    continue
                
                df = pd.read_excel(xls, sheet_name=sheet_name)
                processed_df = process_single_sheet(df, industry_data)
                processed_df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"    处理完成，已写入新文件。")

            except Exception as e:
                print(f"    [错误] 处理Sheet '{sheet_name}' 时发生未知错误，已跳过。")
                print(traceback.format_exc())
                continue
        
        writer.close()
        print(f"\n全部任务成功完成！结果已保存到: {Config.OUTPUT_FILE}")

    except Exception as e:
        print(f"[致命错误] 执行主流程时发生错误。")
        print(traceback.format_exc())

if __name__ == '__main__':
    main()

开始处理文件: E:\重点行业top20_全行业.xlsx
正在加载“行业”Sheet作为计算依据...
加载成功！

开始批量处理各个行业Sheet...
  -> 正在处理Sheet: '化工'


d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
d:\Software\Anaconda\envs\dianli\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning: Conditional Formatting extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\Joe-h\AppData\Local\Temp\ipykernel_14016\2443765303.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = series.replace('\\N', np.nan, regex=False).replace('nan', np.nan)
C:\Users\Joe-h\AppData\Local\Temp\ipykernel_14016\2443765303.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old

    处理完成，已写入新文件。
  -> 正在处理Sheet: '有色'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '建材'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '钢铁'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '电气机械'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '汽车制造'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '计算机'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '金属制品'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '通用设备'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '专用设备'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '仪器仪表'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '铁路船舶'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '医药制造业'
    处理完成，已写入新文件。
  -> 正在处理Sheet: '纺织业'


C:\Users\Joe-h\AppData\Local\Temp\ipykernel_14016\2443765303.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = series.replace('\\N', np.nan, regex=False).replace('nan', np.nan)
C:\Users\Joe-h\AppData\Local\Temp\ipykernel_14016\2443765303.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  s = series.replace('\\N', np.nan, regex=False).replace('nan', np.nan)
C:\Users\Joe-h\AppData\Local\Temp\ipykernel_14016\2443765303.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versi

    处理完成，已写入新文件。

全部任务成功完成！结果已保存到: E:\重点行业top20_结果.xlsx
